<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/ResNeXt101_32x8d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [2]:
%cd /content/gdrive/MyDrive/HatefulMemesDetection/Code/Experiments

/content/gdrive/MyDrive/HatefulMemesDetection/Code/Experiments


In [3]:
!nvidia-smi

Wed Apr 26 17:38:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!sudo apt-get update -y
!sudo apt-get install python3.7
!pip install torch transformers torchvision

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,644 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,170 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,554 kB]
Hit:12 http://archive.ubuntu.com/ubuntu focal InRelease
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu 

In [5]:
'''IMPORTING NECESSARY MODULES
'''
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data

import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/Trainers/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/Dataloaders/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/utils/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/architectures/')

from dataloader import mydataset #, create_prime_dict 
from trainer_ResNet import train, test_classify
from resnet_models import ResNet,Bottleneck,resnext101_32x8d
from Load_model import load
from plot_curves import plot_loss, plot_acc

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


**Dataloading Scheme**

In [7]:
trainlist = '/content/gdrive/MyDrive/hm_project_pe/data/train_o.jsonl'
validlist = '/content/gdrive/MyDrive/hm_project_pe/data/dev_o.jsonl'
img_dir = '/content/gdrive/MyDrive/hm_project_pe/data'

In [8]:
# prime_dict = create_prime_dict(trainlist)

In [9]:
'''
Train Dataloader
''' 
train_dataset = mydataset(annotations_file=trainlist, img_dir=img_dir, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 32, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mydataset(annotations_file=validlist, img_dir=img_dir, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 32, num_workers=16,pin_memory=True)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_ms

**Model Definition**

Train Resnet50/ ResNext101

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [11]:
modelname = 'ResneXt101_32x8d'
modelpath = './saved_model_checkpoints/'+modelname

In [12]:
# model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)


'''
Pretrained ResNeXt101_32x8d
'''
model = resnext101_32x8d(pretrained = True)


model = nn.DataParallel(model,device_ids=[0]).to(device)
model


# '''
# Load saved model from checkpoint
# '''
# model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth
100%|██████████| 340M/340M [00:01<00:00, 345MB/s]


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

In [13]:
'''
Transfer Learning
'''

# for param in model.module.parameters():
#     param.requires_grad = False

fc_inputs = model.module.fc.in_features

model.module.fc = nn.Sequential(
    nn.Linear(fc_inputs, 2)
#     nn.BatchNorm1d(4096),
#     nn.ReLU(),
#     nn.Linear(768, 2)
    
)

model.to(device)

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

**Hyperparameters**

In [14]:
## Loss Function
criterion = nn.CrossEntropyLoss()

lr_list = [0.1, 0.05, 0.001]

for lr in lr_list:

  # Optimizer
  optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=1e-4, momentum=0.9)

  lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 20, gamma = 0.1)

  # Epochs
  num_Epochs = 40

  writer = SummaryWriter("ResNeXt101_32x8d")

  train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)\

  writer.flush()
  writer.close()


loss 1.1508672899007797
loss 1.8231927579641343
Epoch:  1
training loss =  0.8490746377554155
Validation Loss: 0.7194	Top 1 Validation Accuracy: 0.5060
loss 0.6507968670129776
loss 1.3115506947040558
Epoch:  2
training loss =  0.6555788584221575
Validation Loss: 0.7572	Top 1 Validation Accuracy: 0.5060
loss 0.6619167000055313
loss 1.311177109479904
Epoch:  3
training loss =  0.6552066422046575
Validation Loss: 0.7757	Top 1 Validation Accuracy: 0.5040
loss 0.649909947514534
loss 1.30594215631485
Epoch:  4
training loss =  0.6549271858276281
Validation Loss: 0.7288	Top 1 Validation Accuracy: 0.5060
loss 0.6607723993062973
loss 1.3066352355480193
Epoch:  5
training loss =  0.6565449913417486
Validation Loss: 0.7372	Top 1 Validation Accuracy: 0.5040
loss 0.6431145712733268
loss 1.3023745194077492
Epoch:  6
training loss =  0.6531191115316591
Validation Loss: 0.7396	Top 1 Validation Accuracy: 0.5060
loss 0.6491126003861427
loss 1.3057643952965736
Epoch:  7
training loss =  0.652926725664533

In [24]:
'''
Load saved model from checkpoint
'''
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)

**Evaluate**

In [25]:
test_loss, accuracy, original_label, predicted_label, prediction_probs = test_classify(model, validation_dataloader, criterion, device)

In [26]:
import pandas as pd
original = pd.read_json("/content/gdrive/MyDrive/HM_Dev_Results/dev_original.jsonl", lines=True)

In [27]:
original.head()

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,img/80243.png,1,mississippi wind chime


In [28]:
original['original_label'] = original_label
original['predicted_label'] = predicted_label
original['prediction_probs'] = prediction_probs
original.to_csv('Resnet_prediction.csv', index=False)

In [29]:
original.head()

,id,img,label,text,original_label,predicted_label,prediction_probs
0,8291,img/08291.png,1,white people is this a shooting range,1,0,0.710926
1,46971,img/46971.png,1,bravery at its finest,1,1,0.505940
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,1,0,0.682672
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,1,0,0.698179
4,80243,img/80243.png,1,mississippi wind chime,1,0,0.657771


In [21]:
avg_loss = 0

for batch_num, (feats, captions, input_id, attention_masks, target) in enumerate(train_dataloader):
        feats, target = feats.to(device), target.to(device)


        '''
        Compute output and Loss
        '''
        output, embeddings = model(feats) 
        total_loss = criterion(output, target)
        
        
        avg_loss += total_loss.item()
        
        
avg_loss/len(train_dataloader)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0.615032618095104

In [ ]:
'''
Scale of embeddings
'''
embeddings[0][0:100]

In [23]:
'''
Scale of output
'''
output[0]

tensor([-0.3980,  0.3829], device='cuda:0', grad_fn=<SelectBackward0>)

tensor([-0.3980,  0.3829], device='cuda:0', grad_fn=<SelectBackward0>)